# Prerequisites



1.   You labled your images with the VGG Image Annotator (VIA) and only used polygons
2.   You saved the training images and the json file for the training images in data/train (same for the validation set)
3.   You adjusted the load_balloon method in the balloon python file (see the ppt presentation)
4.   If you want to continue training a network, you loaded it in a appropriate directory as a hdf5 file



# Clone the repo

In [0]:
!git clone https://github.com/zhangkaidhb/Mask_RCNN

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 1108, done.
remote: Total 1108 (delta 0), reused 0 (delta 0), pack-reused 1108
Receiving objects: 100% (1108/1108), 123.19 MiB | 37.19 MiB/s, done.
Resolving deltas: 100% (626/626), done.


This code is used to create the repository in which the trained neural net will be safed

In [0]:

import os
os.chdir("./Mask_RCNN/data/gear")

# Confirm that you use the GPU
click on "runtime" -> "change runtime type" -> choose "GPU" on the accelerator field. (The free GPU is why we use colab)

In [0]:
# %tensorflow_version 2.x
# !pip uninstall tf-nightly
# !pip uninstall tensorflow
# !pip uninstall -y tensorflow
# !pip install tensorflow-gpu==1.14.0
%tensorflow_version 1.x   
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Manage imports

In [0]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

from samples.balloon import balloon

%matplotlib inline
%matplotlib inline

Config of paths

In [0]:

config = balloon.BalloonConfig()
BALLOON_DIR = os.path.join(ROOT_DIR, "data/gear")

# Load the dataset

In [0]:
# Load dataset
# Get the dataset from the releases page
# https://github.com/matterport/Mask_RCNN/releases
dataset = balloon.BalloonDataset()
dataset.load_balloon(BALLOON_DIR, "train")


# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

# Visualize the Training images
This Section is used to validate that everything is fine with the training set.

In [0]:

# Load and display random samples
image_ids = np.random.choice(dataset.image_ids, 4)
for image_id in image_ids:
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names)

visualize image boxes

In [0]:

# Load random image and mask.
image_id = random.choice(dataset.image_ids)
image = dataset.load_image(image_id)
mask, class_ids = dataset.load_mask(image_id)
# Compute Bounding box
bbox = utils.extract_bboxes(mask)

# Display image and additional stats
print("image_id ", image_id, dataset.image_reference(image_id))
log("image", image)
log("mask", mask)
log("class_ids", class_ids)
log("bbox", bbox)
# Display image and instances
visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names)

In [0]:
image_id = np.random.choice(dataset.image_ids, 1)[0]
image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
    dataset, config, image_id, use_mini_mask=False)

log("image", image)
log("image_meta", image_meta)
log("class_ids", class_ids)
log("bbox", bbox)
log("mask", mask)

display_images([image]+[mask[:,:,i] for i in range(min(mask.shape[-1], 7))])

In [0]:
visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names)

In [0]:
# Add augmentation and mask resizing.
image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
    dataset, config, image_id, augment=True, use_mini_mask=True)
log("mask", mask)
display_images([image]+[mask[:,:,i] for i in range(min(mask.shape[-1], 7))])

# Train the network

In [0]:

os.chdir("..")
os.chdir("..")
os.chdir("./samples/balloon")

If you have trained the neural net before and want to continue training, change coco to last

In [0]:
!python3 balloon.py train --dataset=/content/Mask_RCNN/data/gear --weights=coco

The trained network will be safed somewhere in the logs directory.
You can browse the directories by clicking on "files" at the left hand side of this website.
After you close your browser your runtime will be disconnected. This implies that all the files you created will be lost. So if you want to keep the trained neural net you will have to download the hdf5 file(right click and download in the file manager)